In [ ]:
# Importar e instalar dependencias
import pandas as pd
import numpy as np
import re
from collections import Counter

# CARGA DE DATOS

In [ ]:
!pip install gdown
# Descarga el archivo del dataset de drive usando gdown
url = 'https://drive.google.com/file/d/1LkEJ3rstkdyhUGWi9O2YKQMe0wC_ZyDd/view?usp=sharing'
file_id = url.split('/')[-2]
!gdown --id $file_id

# Carga el dataset usando pandas
sd = pd.read_csv('/content/Suicide_Detection.csv')

In [ ]:
# Sampleo de data para agilizar el testeo del código
sd = sd.sample(n=10000, random_state=42)

# MODELADO, ENTRENAMIENTO Y EVALUACIÓN

In [ ]:
#Copia del dataset limpio (pre_processed)
pre_processed = sd.copy()

In [ ]:
from sklearn.model_selection import train_test_split

# Separación de dataset en training y testing
train_data ,test_data = train_test_split(pre_processed,test_size=0.2,random_state=10)

print('Training data: ',len(train_data))
print('Testing data: ',len(test_data))


### Modelo de regresión logística basado en TFIDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, classification_report, recall_score, f1_score, confusion_matrix,  classification_report
import pandas as pd

# TF-IDF
vectorizer_tfidf = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2))
X_tfidf_train = vectorizer_tfidf.fit_transform(train_data['text'])
X_tfidf_test = vectorizer_tfidf.transform(test_data['text'])

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300]}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_tfidf_train, train_data['class'])


# Entrenamiento del modelo
model = LogisticRegression(penalty='l2', C=10, solver='saga', max_iter=200)
model.fit(X_tfidf_train, train_data['class'])

best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

model = LogisticRegression(**best_params)
model.fit(X_tfidf_train, train_data['class'])

# Evaluación del modelo
predictions_tfidf = model.predict(X_tfidf_test)
accuracy_tfidf = accuracy_score(test_data['class'], predictions_tfidf)
precision_tfidf = precision_score(test_data['class'], predictions_tfidf, average='weighted')
recall_tfidf = recall_score(test_data['class'], predictions_tfidf, average='weighted')
f1_tfidf = f1_score(test_data['class'], predictions_tfidf, average='weighted')
confusion_mat_tfidf = confusion_matrix(test_data['class'], predictions_tfidf)

print(classification_report(test_data['class'], predictions_tfidf))

print("Accuracy:", accuracy_tfidf)
print("Precision:", precision_tfidf)
print("Recall:", recall_tfidf)
print("F1-Score:", f1_tfidf)
print("Confusion Matrix:\n", confusion_mat_tfidf)

In [ ]:
print("Número de características en TF-IDF:", X_tfidf_train.shape[1])